# Предварительная настройка среды разработки

## Глобальные параметры

Глобальные параметры работы блокнота для упрощения управления процессом обучения:

In [1]:
dataset_name = "mozilla-foundation/common_voice_13_0"          # Название датасета
dataset_lang = "ru"                                            # Язык датасета

model_name = "whisper-small"                                    # Базовая модель
model_task = "transcribe"

tokenizer_language = "Russian"                                 # Язык токенизатора


# Автоматически формируемые параметры
reference_model = "openai/" + model_name
result_model_name = model_name + "-fine_tuned-ru"

# Время начала и конца дешевого тарифа для трехтарифного счётчика
start_time = "23:00:00"
end_time = "07:00:00"

# DEBUG
print(dataset_name, dataset_lang)
print(tokenizer_language, model_task)
print(reference_model)
print(result_model_name)

mozilla-foundation/common_voice_13_0 ru
Russian transcribe
openai/whisper-small
whisper-small-fine_tuned-ru


## Установка необходимых библиотек

### Локальная среда

##### Anaconda

In [2]:
# Установка необходимых библиотек в локальную среду

# !conda install -y -c conda-forge transformers
# !conda install -y -c conda-forge datasets
# !conda install -y -c conda-forge librosa 
# !conda install -y -c conda-forge ffmpeg 
# !conda install -y -c conda-forge huggingface_hub 
# !conda install -y -c conda-forge tensorboard
# !conda install -y -c conda-forge git-lfs
# !pip install --no-input evaluate
# !pip install --no-input jiwer
# !pip install --no-input gradio
# !pip install huggingface_hub["cli"]

print("Local environment setup complete.")

Local environment setup complete.


#### Python Pip

In [3]:
# Тестировалось на WSL 2 Ubuntu 22.04.2 LTS
# ! pip install transformers
# ! pip install datasets
# ! pip install huggingface_hub
# ! pip install huggingface_hub["cli"]
# ! pip install evaluate
# ! pip install gradio
# ! pip install tensorboard
# ! pip install git-lfs
# ! pip install librosa
# ! pip install ffmpeg
# ! pip install jiwer

print("Local environment setup complete.")

Local environment setup complete.


#### Google Colab

In [4]:
# Установка необходимых библиотек для Google Collab

# !add-apt-repository -y ppa:jonathonf/ffmpeg-4
# !apt update
# !apt install -y ffmpeg
# !pip install datasets>=2.6.1
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install evaluate>=0.30
# !pip install jiwer
# !pip install gradio

# Подключим Google Drive
# drive.mount('/content/drive')
# print("Google Drive mounted successfully.")

print("Library installation complete!")

Library installation complete!


## Проверка доступности GPU

Чтобы получить GPU, нажмите _Runtime_ -> _Change runtime type_, затем измените _Hardware accelerator_ с _None_ на _GPU_.

Мы можем проверить, что нам назначен GPU, и просмотреть его характеристики:

In [2]:
!nvidia-smi

Wed Aug 30 11:26:36 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98.01              Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:01:00.0  On |                  Off |
| 31%   32C    P8              42W / 450W |   1431MiB / 24564MiB |     14%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Подключение библиотек

In [3]:
# Вспомогательные библиотеки
import os
import subprocess
import os
import multiprocessing
import warnings
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import time

# HuggingFace
from huggingface_hub import notebook_login, login
from datasets import load_dataset, load_from_disk, DatasetDict
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from datasets import Audio
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers.utils import logging

# Evaluate
import evaluate

# PyTorch
import torch

print("Required libraries/modules initialization complete!")

Required libraries/modules initialization complete!


## Настройка окружения

In [4]:
# Определим количество ядер CPU, это влияет на количество рабочих процессов которые мы можем запустить одновременно
system_num_workers = multiprocessing.cpu_count()
print("The number of workers processes is set at", system_num_workers)

# Отключение назойлевых предупреждений
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Аутентификация ноутбука в HuggingFace HUB
notebook_login()

print("Environment setup completed!")

The number of workers processes is set at 20


Environment setup completed!


## Определение вспомогательных функций

In [5]:
# import datetime
# from time import sleep

# def operates_during_cheap_electricity_tariffs(start_time, end_time):

#     # Функция для проверки укладывается ли текущее время в диапазон дешевого тарифа
#     def in_between(now, start, end):
#         if start <= end:
#             return start <= now < end
#         else: # over midnight e.g., 23:30-04:15
#             return start <= now or now < end

#     # Функций возвращающая текущее время timedelta
#     def get_now():
#         now = (datetime.datetime.now()).time()
#         td_now = datetime.timedelta(hours=now.hour, minutes=now.minute, seconds=now.second)
#         return td_now

#     # Функция конвертирующая время суток в 24-часово формате в timedelta
#     def convert_str_timedelta(time):
#         result = (datetime.datetime.strptime(time, "%H:%M:%S")).time() 
#         result = datetime.timedelta(hours=result.hour, minutes=result.minute, seconds=result.second)
#         return result
    
#     td1 = convert_str_timedelta(start_time)
#     td2 = convert_str_timedelta(end_time)
#     td_now = get_now()

#     if not in_between(td_now, td1, td2):
#         time_wait = (td1 - td_now)
#         print("Current time:", td_now ," Work has been suspended until the time of the cheaper electricity tariff at", start_time)
#         sleep(time_wait.total_seconds())

#         td_now = get_now()
#         print("Current time:", td_now ,"Work resumed.")

# print("Вспомогательные функции определены.")

Вспомогательные функции определены.


# Тонкая настройка Whisper для многоязыкового ASR с помощью библиотеки 🤗 Transformers

В этом блокноте мы представляем пошаговое руководство по тонкой настройке Whisper для любого многоязыкового набора данных ASR с помощью библиотеки Hugging Face 🤗 Transformers. Это более "практическая" версия сопутствующего [сообщения в блоге](https://huggingface.co/blog/fine-tune-whisper). Для более подробного объяснения Whisper, набора данных Common Voice и теории, лежащей в основе тонкой настройки, читателю рекомендуется обратиться к статье в блоге.

## Введение

Whisper - это предварительно обученная модель для автоматического распознавания речи (ASR) опубликованная в [сентябре 2022](https://openai.com/blog/whisper/) авторами Алеком Рэдфордом и другими из OpenAI. В отличие от многих своих предшественников, таких как. 
[Wav2Vec 2.0](https://arxiv.org/abs/2006.11477), которые предварительно обученны на неразмеченных аудиоданных, Whisper предварительно обучен на огромном количестве **размеченных** транскрибированных аудио данных, 680 000 часов, если быть точным. Это на порядок больше данных, чем использованные для обучения Wav2Vec 2.0 (60 000 часов) неразмеченных аудиоданных. Более того, 117 000 часов из этих данных использованных для предварительного обучения - это многоязыковые данные ASR. Это позволяет получить контрольные точки которые могут быть применены к более чем 96 языкам, многие из которых считаются _низкоресурсными_.

При масштабировании на 680 000 часов размеченных данных предварительного обучения, модели Whisper демонстрируют высокую способность к обобщению для многих наборов данных и областей (доменов). Предварительно обученные контрольные точки достигают результатов, конкурентоспособных с современными 
ASR системами, с коэффициентом ошибок в словах (WER) около 3% на подмножестве чистых тестов LibriSpeech ASR и новым передовым результатом на TED-LIUM с 4,7% (смотрите таблицу 8 из [Whisper paper](https://cdn.openai.com/papers/whisper.pdf)). 

Обширные знания о многоязычном ASR, полученные Whisper во время предварительного обучения могут быть использованы для других языков с низким уровнем ресурсов; посредством тонкой настройки предварительно обученные контрольные точки могут быть адаптированы для конкретных наборов данных и языков для дальнейшего улучшения результатов. Мы покажем, как можно точно настроить Whisper для языков с ограниченными ресурсами в этом блокноте.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Рисунок 1:</b> модель Whisper. Архитектура соответствует стандартной модели кодера-декодера на основе трансформатора. Log-Mel спектрограмма подается на вход кодера. Последние скрытые состояния поступают в декодер через механизмы перекрестного внимания. Декодер авторегрессионно предсказывает текстовые токены, совместно обусловленные скрытыми состояниями энкодера и ранее предсказанными токенами. Источник рисунка: 
<a href="https://openai.com/blog/whisper/">блог OpenAI Whisper</a>.</figcaption>
</figure>

Контрольные точки Whisper представлены в пяти конфигурациях с различными размерами моделей. Самые маленькие четыре модели обучаются либо только на английском, либо на многоязычных данных. Самая большая контрольная точка - только многоязычная. 
Все девять предварительно обученных контрольных точек доступны на сайте [Hugging Face Hub](https://huggingface.co/models?search=openai/whisper). Контрольные точки сведены в следующую таблицу со ссылками на модели в хабе:

| Размер   | Количество слоёв | Ширина | Количество голов | Параметры | Только англоязычная модель                                         | Многоязычная модель                                      |
|--------|--------|-------|-------|------------|------------------------------------------------------|---------------------------------------------------|
| tiny   | 4      | 384   | 6     | 39 M       | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)  |
| base   | 6      | 512   | 8     | 74 M       | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)   |
| small  | 12     | 768   | 12    | 244 M      | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)  |
| medium | 24     | 1024  | 16    | 769 M      | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium) |
| large  | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large)  |

В демонстрационных целях мы доработаем контрольную точку многоязычной версии [``малой модели``](https://huggingface.co/openai/whisper-small) с 244M параметрами (~= 1GB). Что касается наших данных, мы обучим и оценим нашу систему на языке с низким уровнем ресурсов, взятом из набора данных [Common Voice](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Мы покажем, что всего за 8 часов тонкой настройки данных мы можем добиться высоких результатов на этом языке.

------------------------------------------------------------------------

${}^1$ - Название Whisper происходит от аббревиатуры "WSPSR", которая расшифровывается как "Web-scale Supervised Pre-training for Speech Recognition".

## Загрузка датасета

Используя 🤗 Datasets, загрузка и подготовка данных чрезвычайно проста. 

Мы можем загрузить и подготовить сплиты Common Voice всего одной строкой кода. 

Во-первых, убедитесь, что вы приняли условия использования на Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). После принятия условий вы получите полный доступ к набору данных и сможете загружать данные локально.

Объединим `train` и `validation` части набора данных, чтобы получить как можно больше данных для обучения. Будем использовать
данных `test` в качестве нашего тестового набора.

> ВАЖНО!
> Для удобства хранения данных датасета, я ранее определил путь к папке в переменной `DATA_PATH`. Это позволит хранить кеш данных в известном заранее месте и в дальшнейшем удалить ненужные данные. Это позволит быстро освободить место на HDD.

In [5]:
common_voice =  load_dataset("artyomboyko/common_voice_13_0_ru_dataset_for_whisper_fine_tune")

common_voice

Found cached dataset parquet (/home/artyom/.cache/huggingface/datasets/artyomboyko___parquet/artyomboyko--common_voice_13_0_ru_dataset_for_whisper_fine_tune-42b90f359d1e87c7/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 36454
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 10186
    })
})

In [ ]:
# Создадим словарь для хранения датасета
common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name, dataset_lang, split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset(dataset_name, dataset_lang, split="test", use_auth_token=True)

print(common_voice)
print("Data download successfully completed!")

Большинство наборов данных ASR предоставляют только входные образцы аудио (`audio`) и соответствующий транскрибированный текст (`sentence`).  Common Voice содержит дополнительную метаданные, такие как `accent` и `locale`, которые мы можем игнорировать для ASR.

Оставив блокнот максимально общим, мы рассматриваем только входной звук и транскрибированный текст для тонкой настройки, отбрасывая дополнительную информацию метаданных:

In [21]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'variant'],
        num_rows: 36454
    })
    test: Dataset({
        features: ['audio', 'sentence', 'variant'],
        num_rows: 10186
    })
})


## Подготовка экстрактора признаков, токенизатора и данных

Конвейер ASR можно разделить на три этапа: 
1) экстрактор признаков, который предварительно обрабатывает необработанные аудиоданные
2) Модель, которая выполняет сопоставление последовательности с последовательностью 
3) Токенизатор, который постобрабатывает выводы модели в текстовый формат.

Модель Whisper из библиотеки 🤗 Transformers имеет ассоциированный экстрактор признаков и токенизатор, называемые [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
и [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer) 
соответственно.

Рассмотрим детальнее настройки экстрактора и токенизатора по очереди!

### Загрузка WhisperFeatureExtractor

Экстрактор признаков Whisper выполняет две операции:
1. Разбивка / усечение аудиоданных до 30 секунд: все аудиоданные короче 30 секунд разбиваются на молчание (нули), а те, что длиннее 30 секунд, усекаются до 30 секунд.
2. Преобразует входные аудио в _log-Mel спектрограммы_ входных признаков, визуальное представление аудио и форму входных данных, ожидаемых моделью Whisper.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Рисунок 2:</b> Преобразование дискретизированного аудио массива в log-Mel спектрограмму.
Слева: дискретизированный одномерный аудиосигнал. Справа: соответствующая log-Mel спектрограмма. Источник рисунка:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

Мы загрузим экстрактор признаков из предварительно обученной контрольной точки со параметрами по умолчанию:

In [6]:
# Переопределим модель для проведения дообучения 
reference_model = "artyomboyko/whisper-small-fine_tuned-ru"

In [7]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(reference_model)

print("The feature extractor is loaded successfully.")

The feature extractor is loaded successfully.


### Загрузка WhisperTokenizer

Модель Whisper выдает последовательность идентификаторов _токенов_. Токенизатор сопоставляет каждый из этих идентификаторов токенов с соответствующей текстовой строкой. Для русского языка мы можем загрузить предварительно обученный токенизатор и использовать его для тонкой настройки 
с указанием целевого языка и задачи. Эти аргументы сообщают токенизатору о том, что следует задавать префикс токена языка и задачи в начале последовательности меток:

In [8]:
tokenizer = WhisperTokenizer.from_pretrained(reference_model,
                                             language = tokenizer_language,
                                             task = model_task)

print("Tokenizer loaded successfully.")

Tokenizer loaded successfully.


### Комбинирование для создания WhisperProcessor

Чтобы упростить использование экстрактора признаков и токенизатора, мы можем _обернуть_ их в один класс `WhisperProcessor`. Этот объект наследуется от `WhisperFeatureExtractor` и `WhisperProcessor`, и может использоваться для обработки входных аудиоданных и 
получения предсказаний модели по мере необходимости. При этом во время обучения нам нужно отслеживать только два объекта: `processor` и `model`:

In [9]:
processor = WhisperProcessor.from_pretrained(reference_model, 
                                             language = tokenizer_language,
                                             task = model_task)

print("Processor loaded successfully.")

Processor loaded successfully.


### Подготовка данных

Давайте распечатаем первый образец набора данных Common Voice, чтобы посмотреть в какой форме находятся данные:

In [ ]:
print(common_voice["train"][0])

Поскольку наш входной звук сэмплирован с частотой 48 кГц (`'...sampling_rate': 48000...`), нам нужно _уменьшить_ его до частоту дискретизации до 16 кГц перед тем, как передать его в экстрактор признаков Whisper, 16 кГц - это частота дискретизации, ожидаемая моделью Whisper. Мы установим для аудио входов правильную частоту дискретизации с помощью метода [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column). Эта операция не изменяет звук "на месте", а скорее дает сигнализирует `datasets` передискретизировать аудиосэмплы _на лету_ при при первой загрузке данных:

In [33]:
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print("Audio sample rate set successfully.")

Audio sample rate set successfully.


Повторная загрузка первого аудиообразца из набора данных Common Voice приведет к его передискретизации до нужной частоты дискретизации:

In [34]:
print(common_voice["train"][0])

{'audio': {'path': '/home/artyom/.cache/huggingface/datasets/downloads/extracted/a89420fafbe263300e50be6d61d5d75633b434150dc09fe5f7cde741f4306e05/ru_train_0/common_voice_ru_22070781.mp3', 'array': array([ 2.91038305e-11, -2.91038305e-11,  8.00355338e-11, ...,
        1.80914412e-08,  8.51605364e-09, -5.54000508e-08]), 'sampling_rate': 16000}, 'sentence': 'Я рад, что Генеральная Ассамблея начала ее обсуждение.', 'variant': ''}


Теперь мы можем написать функцию для подготовки наших данных к работе с моделью:
1. Мы загружаем и передискретизируем аудиоданные, вызывая `batch["audio"]`. Как объяснялось выше, 🤗 Datasets выполняет все необходимые операции по передискретизации на лету.
2. Мы используем экстрактор признаков для вычисления входных признаков спектрограммы log-Mel из нашего одномерного аудио массива.
3. Мы кодируем транскрипции в идентификаторы меток с помощью токенизатора.

In [35]:
def prepare_dataset(batch):
    # Загрузим и передискретизируем аудиоданные с 48 до 16 кГц
    audio = batch["audio"]

    # вычислим log-Mel входные признаки из входного аудио массива  
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # закодируем целевой текст в идентификаторы меток
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

Мы можем применить функцию подготовки данных ко всем нашим учебным примерам, используя метод `.map` в наборе данных. Аргумент `num_proc` указывает, сколько ядер процессора использовать. Если задать `num_proc` > 1, то будет включена многопроцессорная обработка. Если метод `.map` зависает при многопроцессорной обработке, установите `num_proc=1` и обрабатывайте набор данных последовательно.

> ВАЖНО!    
> Для удобства работы я использую автоматическое определения количества рабочих процессов через переменную `system_num_workers`. Это позволяет существенно повысить производительность и сэкономить время.

In [36]:
print("The current number of CPU cores is {}.".format(system_num_workers))

# common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc = system_num_workers)

common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc = 1)

The current number of CPU cores is 20.


Map:   0%|          | 0/36454 [00:00<?, ? examples/s]

Map:   0%|          | 0/10186 [00:00<?, ? examples/s]

#### Сохранение обработанных данных

Чтобы не тратить время на предварительную подготовку данных, созраним подготовленный датасет на диск:

In [16]:
!mkdir -v "PREPARED_DATASET"

prepared_dataset_path = "./PREPARED_DATASET"

common_voice.save_to_disk(prepared_dataset_path)

#### Загрузка ранее предобработанных данных

Загрузим ранее предобработанный датасет:

In [18]:
prepared_dataset_path = "./PREPARED_DATASET"

common_voice = load_from_disk(prepared_dataset_path)

In [ ]:
print(common_voice["train"][0])

## Обучение и оценка

Теперь, когда мы подготовили наши данные, мы готовы погрузиться в конвейер обучения. [🤗Тренер (Trainer)](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) сделает за нас большую часть тяжелой работы. Все, что нам нужно сделать, это:
- Определить коллатор данных (data collator): коллатор данных берет наши предварительно обработанные данные и подготавливает тензоры PyTorch, готовые для модели.
- Выбрать метрику для оценки: во время оценки мы хотим оценить модель с помощью метрики [word error rate (WER)](https://huggingface.co/metrics/wer). Нам нужно определить функцию `compute_metrics`, которая будет обрабатывать эти вычисления.
- Загрузить предварительно обученную контрольную точку: нам нужно загрузить предварительно обученную контрольную точку и правильно настроить ее для обучения.
- Определить конфигурацию процесса обучения: она будет использоваться 🤗 тренером для определения параметров тренировок.

После того как мы произведем тонкую настройку модели, мы оценим ее на тестовых данных, чтобы убедиться, что мы правильно обучили ее транскрибировать речь.

### Определение коллатора данных

Коллатор данных для модели последовательной речи уникален в том смысле, что он обрабатывает входные признаки `input_features` и метки `labels` независимо друг от друга: входные признаки `input_features` должны быть обработанны экстрактором признаков, а метки `labels` - токенизатором. 

Входные признаки `input_features` уже разбиты на аудиофрагменты длительностью 30 секунд и преобразованы в соответствующие им спектрограммы log-Mel фиксированной размерности с помощью экстрактора признаков, поэтому все, что нам нужно сделать, это преобразовать входные признаки `input_features` в тензоры PyTorch. Мы делаем это с помощью метода `.pad` экстрактора признаков с хаданием параметра `return_tensors=pt`.

С другой стороны, `метки (labels)` не имеют вставок (un-padded). Сначала мы выравниваем последовательности до максимальной длины в батче, используя метод `.pad` токенизатора. Затем токены-вставки заменяются на значение `-100`, чтобы эти токены **не** учитывались при вычислении потерь. Затем мы вырезаем токен BOS из начала последовательности меток, так как мы добавим его позже во время обучения. 

Мы можем использовать препроцессор `WhisperProcessor`, который мы определили ранее, для выполнения извлечения признаков, так и для работы с токенами:

In [10]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # разделить входы и метки, так как они должны быть разной длины и нуждаться в разных методах дополнения (padding)
        # сначала обрабатываем аудиовходы, просто возвращая тензоры torch
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # получим токенизированные последовательности меток
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # заменим дополнение (padding) на -100, чтобы корректно игнорировать потери
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # если токен bos был добавлен на предыдущем шаге токенизации,
        # вырезаем здесь токен bos, так как он все равно будет добавлен позже
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Давайте инициализируем коллатор данных, который мы только что определили:

In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Метрика оценки

Мы будем использовать метрику коэффициента ошибок в словах (WER), которая является "де-факто" метрикой для оценки систем ASR-систем. За дополнительной информацией обратитесь к документу WER [docs](https://huggingface.co/metrics/wer). Мы загрузим метрику WER из 🤗 Evaluate:

In [12]:
metric = evaluate.load("wer")
#metric_cer = evaluate.load("cer")

print("Metric is loaded successfully.")

Metric is loaded successfully.


Затем нам просто нужно определить функцию, которая принимает предсказания нашей модели и возвращает метрику WER. Эта функция, называемая `compute_metrics`, сначала заменяет `-100` на `pad_token_id` в `label_ids` (отменяя шаг, который мы применили в 
в коллаторе данных, чтобы правильно игнорировать подстановочные токены в потерях). Затем она декодирует прогнозы и идентификаторы меток в строки. Наконец, она вычисляет WER между прогнозами (predictions) и эталонными метками (reference labels):

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # заменяем -100 на pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # мы не хотим группировать токены при вычислении метрики
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    #cer = 100 * metric_cer.compute(predictions=pred_str, references=label_str)

    #operates_during_cheap_electricity_tariffs(start_time, end_time)

    #return {"wer": wer, "cer": cer}
    return {"wer": wer}


print("The compute_metrics function was defined successfully.")

The compute_metrics function was defined successfully.


### Загрузка контрольной точки предварительно обученной модели

Теперь давайте загрузим предварительно обученную контрольную точку модели Whisper `small`. Опять же, это тривиально благодаря использованию библиотеки 🤗 Transformers!

In [14]:
reference_model = "artyomboyko/whisper-small-fine_tuned-ru"

In [15]:
print("Current reference model:", reference_model)
model = WhisperForConditionalGeneration.from_pretrained(reference_model)

print("Checkpoint of the pre-trained model has been successfully loaded.")

Current reference model: artyomboyko/whisper-small-fine_tuned-ru


Checkpoint of the pre-trained model has been successfully loaded.


Переопределение аргументов генерации - никакие токены не выдаются принудительно в качестве выходов декодера (смотрите [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), никакие токены не подавляются во время генерации (дополнительно смотрите [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [16]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Определение конфигурации обучения

На последнем этапе мы определяем все параметры, связанные с процессом обучения. Более подробную информацию об аргументах процесса обучения можно найти в разделе Seq2SeqTrainingArguments [документации](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

> **Примечание**: если вы не хотите загружать контрольные точки модели в хаб HunggingFace, установите параметр `push_to_hub=False`.

> **ПРИМЕЧАНИЕ**:
> Здесь я разделяю настройки тренера для модели на два варианта:
> - Локальная среда, используется для обучения моделей на локальном ПК,
> - Google Colab, так как в нем доступны GPU с большим обьемом VRAM бесплатно,
>
> ***Оригиниальные настройки обучения модели сохранены в разделе Google Colab!***

#### Локальная среда

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir = result_model_name,      # измените имя репозитория по своему усмотрению
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,       # увеличивается в 2x раза при каждом уменьшении размера батча в 2x раза  
    learning_rate=2e-7,
    warmup_steps=250,
    max_steps=10000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    optim = "adamw_torch",
    per_device_eval_batch_size=8,        # Самый простой способ задать равным параметру per_device_train_batch_size
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

#### Google Colab

In [18]:
# from transformers import Seq2SeqTrainingArguments

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper-base-ru",       # измените имя репозитория по своему усмотрению
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=1,         # увеличивается в 2x раза при каждом уменьшении размера батча в 2x раза
#     learning_rate=1e-6,
#     warmup_steps=500,
#     max_steps=4000,
#     gradient_checkpointing=True,
#     fp16=True,
#     evaluation_strategy="steps",
#     per_device_eval_batch_size=8,          # Самый простой способ задать равным параметру per_device_train_batch_size
#     predict_with_generate=True,
#     generation_max_length=225,
#     save_steps=1000,
#     eval_steps=1000,
#     logging_steps=25,
#     report_to=["tensorboard"],
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
#     push_to_hub=True,
# )

### Задание настроек процесса обучения

Теперь мы можем передать аргументы обучения в 🤗 Trainer вместе с нашей моделью, набором данных, коллатором данных и функцией `compute_metrics`:

In [19]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/artyomboyko/whisper-small-fine_tuned-ru into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/922M [00:00<?, ?B/s]

Download file runs/Aug28_18-20-53_MSK-PC-01/events.out.tfevents.1693236329.MSK-PC-01.744.0:  82%|########2 | 3…

Clean file runs/Aug28_18-20-53_MSK-PC-01/events.out.tfevents.1693236329.MSK-PC-01.744.0:   3%|2         | 1.00…

Download file runs/Aug16_08-45-51_MSK-PC-01/events.out.tfevents.1692164873.MSK-PC-01.484.0:  23%|##2       | 8…

...

Clean file training_args.bin:  25%|##5       | 1.00k/4.00k [00:00<?, ?B/s]

Download file runs/Aug29_13-46-17_MSK-PC-01/events.out.tfevents.1693305992.MSK-PC-01.740.0:  60%|#####9    | 1…

Clean file runs/Aug29_13-46-17_MSK-PC-01/events.out.tfevents.1693305992.MSK-PC-01.740.0:   4%|3         | 1.00…

Clean file model.safetensors:   0%|          | 1.00k/922M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/922M [00:00<?, ?B/s]

### Обучение

Обучение займет примерно 5-10 часов в зависимости от вашего GPU или того, какой GPU был выделен для этого Google Colab. Если вы используете Google Colab непосредственно для тонкой настройки модели Whisper, вы должны убедиться, что обучение не будет прервано из-за бездействия. 
Простым обходным решением для предотвращения этого является вставить следующий код в консоль этой вкладки (_нажать правую кнопку мыши_ -> _обзор_ -> вкладка _консоль_ -> _вставить код_).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton, 60000);
```

Пиковое потребление памяти GPU для данной конфигурации обучения составляет примерно 15,8 ГБ. 
В зависимости от GPU, выделенного в Google Colab, возможно, что при запуске обучения вы столкнетесь с ошибкой CUDA `"out-of-memory"`. 
В этом случае вы можете уменьшить `per_device_train_batch_size` постепенно в 2 раза и использовать [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
для компенсации.

Чтобы запустить обучение, просто выполните:

In [20]:
print("Current model: ", reference_model)
print("Current dataset: ")
trainer.train()
# trainer.train(resume_from_checkpoint=True)
# trainer.train("/home/artyom/Whisper_Train/whisper-small-fine_tuned-ru/checkpoint-3500")

Current model:  artyomboyko/whisper-small-fine_tuned-ru
Current dataset: 


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
500,0.028100,0.221861,17.939589
1000,0.028200,0.222535,18.022201


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Наш лучший WER составляет 32.0% - неплохо для 8 часов обучения! Мы можем отправить нашу контрольную точку в [`hf-speech-bench`](https://huggingface.co/spaces/huggingface/hf-speech-bench) на push, задав соответствующие аргументы ключевых слов (kwargs):

In [28]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_13_0",
    "dataset": "Common Voice 13.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ru, split: test",
    "language": "ru",
    "model_name": "Whisper Small Ru - Artyom Boyko",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [29]:
# kwargs = {
#     "dataset_tags": dataset_name,
#     "dataset": dataset_name.split("/")[1],  # a 'pretty' name for the training dataset
#     "dataset_args": "config: ru, split: test",
#     "language": dataset_lang,
#     # "model_name": "Whisper Base Ru - Artyom Boyko",  # a 'pretty' name for our model
#     "finetuned_from": reference_model,
#     "tasks": "automatic-speech-recognition",
#     "tags": "hf-asr-leaderboard",
# }

Теперь результаты обучения можно загрузить в хаб. Для этого выполните команду `push_to_hub` и сохраните созданный нами объект препроцессора:

In [30]:
trainer.push_to_hub()

## Загрузка выбранной контрольной точки в хаб (не обязательный шаг)

Зададим путь в хабе для сохранения выбранной модели:

In [31]:
hf_hub_path = "artyomboyko/whisper-small-fine_tuned-ru"

Загрузим в память выбранную по окончании процесса обучения модель:

In [32]:
model = WhisperForConditionalGeneration.from_pretrained("./whisper-small-fine_tuned-ru/checkpoint-3500/")
print("Model loaded successfully.")

Model loaded successfully.


Загруим в хаб модель из выбранной контрольной точки:

In [33]:
model.push_to_hub("artyomboyko/whisper-small-fine_tuned-ru")

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/artyomboyko/whisper-small-fine_tuned-ru/commit/fe677ff9eff3682bd06d2c39ed3cfa04be81ddfd', commit_message='Upload WhisperForConditionalGeneration', commit_description='', oid='fe677ff9eff3682bd06d2c39ed3cfa04be81ddfd', pr_url=None, pr_revision=None, pr_num=None)

> **ПРИМЕЧАНИЕ**:
> ***Если загрузить токенизатор в хаб, может сломаться демо модели на странице модели в хабе.***    
> ***НО! Если этого не сделать, то не сможет корректно работать демонстрация из-за невозможности загрузить токенизатор в конвеер.***
> ***Решение этой проблемы - использование токенизатора исходной моедли "openai/whisper-base".***

Сохраним токенизатор в хаб:

In [22]:
tokenizer.push_to_hub(hf_hub_path)

CommitInfo(commit_url='https://huggingface.co/artyomboyko/whisper-small-fine_tuned-ru/commit/6c4187ddf6a45fb4a6b34e9bb5830df4ab8a04a0', commit_message='Upload tokenizer', commit_description='', oid='6c4187ddf6a45fb4a6b34e9bb5830df4ab8a04a0', pr_url=None, pr_revision=None, pr_num=None)

Созраним в хаб препроцессор:

In [24]:
processor.push_to_hub(hf_hub_path)

CommitInfo(commit_url='https://huggingface.co/artyomboyko/whisper-small-fine_tuned-ru/commit/072681218c584b63ef9254b038f73dd50302f5f8', commit_message='Upload processor', commit_description='', oid='072681218c584b63ef9254b038f73dd50302f5f8', pr_url=None, pr_revision=None, pr_num=None)

## Создание демонстрации

Временно переключим режим логирования. Будем выводить только сообщения об ошибках и предупреждения. Это существенно сократит вывод в ячейках ниже.

In [76]:
logging.set_verbosity_warning()

Теперь, когда мы доработали нашу модель, мы можем создать демонстрационный образец, чтобы продемонстрировать ее возможности ASR! 
Мы будем использовать конвейере 🤗 Transformers `pipeline`, который позаботится обо всем, начиная с предварительной обработки аудиовходов и заканчивая декодированием предсказаний модели.

In [77]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="ElectricSecretAgent/whisper-base-fine_tuned-ru", tokenizer="openai/whisper-base")  # необходимо изменить на "ваше_имя_пользователя/имя_модели"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Base - Artyom Boyko",
    description="Real-time demonstration for speech recognition in Russian using a finely tuned base Whisper model.",
)

iface.launch()

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


/home/artyom/miniconda3/envs/pytorch-1.13.1/lib/python3.10/site-packages/gradio/processing_utils.py:236: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/home/artyom/miniconda3/envs/pytorch-1.13.1/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Вернем режим логирования "по умолчанию" - выводим сообщения об ошибках, предупреждениях и основную информацию:

In [74]:
logging.set_verbosity_info()

## Заключительные замечания

В этом блоге мы рассмотрели пошаговое руководство по тонкой настройке Whisper для многоязыкового ASR используя 🤗 Datasets, Transformers и хаб Hugging Face. 
Для получения более подробной информации о модели Whisper, наборе данных Common Voice и теории, лежащей в основе тонкой настройки, обратитесь к сопроводительной записи [в блоге](https://huggingface.co/blog/fine-tune-whisper).
Если вы заинтересованы в тонкой настройке других моделей Transformers, как для английского, так и для многоязычного ASR, обязательно ознакомьтесь с примерами скриптов в [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition).

# Полезные ссылки

- [Fine-Tune Whisper For Multilingual ASR with Transformers (Статья)](https://huggingface.co/blog/fine-tune-whisper)
- [HuggingFace Logging](https://huggingface.co/docs/transformers/main_classes/logging)
- [How to show the learning rate during training](https://discuss.huggingface.co/t/how-to-show-the-learning-rate-during-training/13914/5)